<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [2]:
# Armamos el dataset con cada capitulo siendo un documento
import re
import unicodedata
dataset_path = 'sinceramente.txt'
temp_dataset = []
with open(dataset_path, 'r') as f:
    lines = f.readlines()
    for idx, i in enumerate(lines):
        # Si es una linea en blanco, la ignoro
        pattern = r'^\s*\n'
        match = re.search(pattern, i, flags=re.DOTALL)
        if(match == None):
            temp_dataset.append(i.strip())
            
chapter_idx = []
for idx, line in enumerate(temp_dataset):
    # Ahora, obtengo el num de linea de cada capitulo
    pattern = r'–[0-9].*-*'
    match = re.match(pattern, line)
    if(match != None):
        # print('Found chapter: ', match, f'Line num = {idx}')
        chapter_idx.append(idx)
print(chapter_idx)

import itertools
corpus = []
for chapter_num, idx in enumerate(chapter_idx):
    chapter_start = idx+2
    if(chapter_num < len(chapter_idx)-1):
        chapter_end = chapter_idx[chapter_num+1]-1
    else:
        chapter_end = len(temp_dataset)
    chapter = ''
    print(f'Chapter {chapter_num}: from {chapter_start}-{chapter_end}')
    # Recorro cada capitulo
    for line in range(chapter_start, chapter_end):
        chapter = chapter + temp_dataset[line]
    
    corpus.append(chapter)

print(corpus)

print(f'\nNumero de capitulos: {len(corpus)}')
    # chapter_linenum = []
    # # Ahora, obtengo el num de linea de cada capitulo
    # for idx, i in enumerate(lines):
    #     # Creo regex para encontrar el inicio de cada capitulo
    #     pattern = r'.–[0-9].*-*'
    #     # pattern = re.compile(pattern)
    #     match = re.match(pattern, i)
    #     if(match != None):
    #         print('Found chapter: ', match, f'Line num = {idx}')
    #         chapter_linenum.append(idx)
    # print(chapter_linenum)

[0, 7, 59, 101, 127, 223, 308, 336, 374, 438, 498]
Chapter 0: from 2-6
Chapter 1: from 9-58
Chapter 2: from 61-100
Chapter 3: from 103-126
Chapter 4: from 129-222
Chapter 5: from 225-307
Chapter 6: from 310-335
Chapter 7: from 338-373
Chapter 8: from 376-437
Chapter 9: from 440-497
Chapter 10: from 500-512
['Ayer terminé el último capítulo de este libro y hoy, 12 de marzo de 2019, empiezo a escribir el primero. Estoy en mi departamento de Buenos Aires, en Juncal y Uruguay, pleno barrio de Recoleta. En el mismo lugar desde donde salí con Néstor rumbo a la quinta presidencial de Olivos un 29 de mayo del 2003. El jueves a la madrugada debo viajar a Cuba. Allí se encuentra mi hija, Florencia. Flor, quien producto de la persecución mediática y judicial feroz a la que fue sometida, empezó hace ya un tiempo a tener severos problemas de salud. El brutal estrés que sufrió devastó su cuerpo y sus emociones… Es que es muy terrible para una joven ser acusada de haber ingresado a una asociación ilí

In [3]:
# Ahora divido el corpus en oraciones para procesar cada una individualmente
sentence_tokens = []
for i in corpus:
    sent = i.split('.')
    for j in range(len(sent)):
        # Remuevo caracterez especiales
        sentence_tokens.append(sent[j])

print('Cantidad de oraciones: ', len(sentence_tokens))
print(sentence_tokens[0:10])

Cantidad de oraciones:  6026
['Ayer terminé el último capítulo de este libro y hoy, 12 de marzo de 2019, empiezo a escribir el primero', ' Estoy en mi departamento de Buenos Aires, en Juncal y Uruguay, pleno barrio de Recoleta', ' En el mismo lugar desde donde salí con Néstor rumbo a la quinta presidencial de Olivos un 29 de mayo del 2003', ' El jueves a la madrugada debo viajar a Cuba', ' Allí se encuentra mi hija, Florencia', ' Flor, quien producto de la persecución mediática y judicial feroz a la que fue sometida, empezó hace ya un tiempo a tener severos problemas de salud', ' El brutal estrés que sufrió devastó su cuerpo y sus emociones… Es que es muy terrible para una joven ser acusada de haber ingresado a una asociación ilícita el 27 de octubre del 2010, justo el día de la muerte de su padre, por una situación que ni ella ni ninguna persona que pierde a un padre elige: en Argentina los hijos se convierten en herederos forzosos de su padre por la ley, no porque quieren', ' El 5 de

In [4]:
import nltk
# Descargar el diccionario
nltk.download("punkt")
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
# sentences = [nltk.tokenize.word_tokenize(sent) for sent in sentence_tokens]
sentences = [tokenizer.tokenize(sent.lower()) for sent in sentence_tokens]
print(sentences)

[['ayer', 'terminé', 'el', 'último', 'capítulo', 'de', 'este', 'libro', 'y', 'hoy', '12', 'de', 'marzo', 'de', '2019', 'empiezo', 'a', 'escribir', 'el', 'primero'], ['estoy', 'en', 'mi', 'departamento', 'de', 'buenos', 'aires', 'en', 'juncal', 'y', 'uruguay', 'pleno', 'barrio', 'de', 'recoleta'], ['en', 'el', 'mismo', 'lugar', 'desde', 'donde', 'salí', 'con', 'néstor', 'rumbo', 'a', 'la', 'quinta', 'presidencial', 'de', 'olivos', 'un', '29', 'de', 'mayo', 'del', '2003'], ['el', 'jueves', 'a', 'la', 'madrugada', 'debo', 'viajar', 'a', 'cuba'], ['allí', 'se', 'encuentra', 'mi', 'hija', 'florencia'], ['flor', 'quien', 'producto', 'de', 'la', 'persecución', 'mediática', 'y', 'judicial', 'feroz', 'a', 'la', 'que', 'fue', 'sometida', 'empezó', 'hace', 'ya', 'un', 'tiempo', 'a', 'tener', 'severos', 'problemas', 'de', 'salud'], ['el', 'brutal', 'estrés', 'que', 'sufrió', 'devastó', 'su', 'cuerpo', 'y', 'sus', 'emociones', 'es', 'que', 'es', 'muy', 'terrible', 'para', 'una', 'joven', 'ser', 'ac

[nltk_data] Downloading package punkt to /Users/marcosb/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 2 - Crear los vectores (word2vec)

In [5]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [6]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(sentences=sentences,
                     min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=10,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [7]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 6026


In [8]:
# Cantidad de words encontradas en el corpus
words = list(w2v_model.wv.index_to_key)
print(words)
print("Cantidad de words distintas en el corpus:", len(words))

['de', 'la', 'que', 'y', 'el', 'en', 'a', 'los', 'no', 'por', 'del', 'con', 'un', 'se', 'las', 'una', 'lo', 'para', 'me', 'como', 'es', 'había', 'al', 'más', 'su', 'era', 'fue', 'le', 'cuando', 'ese', 'néstor', 'mi', 'o', 'gobierno', 'pero', 'porque', 'también', 'argentina', 'muy', 'todo', 'estaba', 'sin', 'durante', 'esa', 'sus', 'nos', 'sobre', 'país', 'yo', 'si', 'todos', 'después', 'qué', 'ser', 'años', 'nuestro', 'eso', 'ya', 'sólo', 'vez', 'él', 'ni', 'año', 'habían', 'siempre', 'nunca', 'dos', 'entre', 'desde', 'sino', 'presidente', 'poder', 'día', 'estado', 'así', 'sido', 'tenía', 'ley', 'contra', 'argentinos', 'mismo', 'macri', 'medios', 'hasta', 'además', 'momento', 'dijo', 'casa', 'habíamos', 'política', 'dije', 'sí', 'este', 'nacional', 'donde', 'recuerdo', 'millones', 'entonces', 'les', 'lugar', 'son', 'antes', 'santa', 'judicial', 'historia', 'acuerdo', 'ciento', 'embargo', 'clarín', 'mucho', 'ellos', 'e', '2015', 'causa', 'nuestra', 'otro', 'cosas', 'uno', 'parte', 'habe

### 3 - Entrenar el modelo generador

In [9]:
from timeit import default_timer as timer

st = timer()

# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentences,
                 total_examples=w2v_model.corpus_count,
                 epochs=1000,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

et = timer()
res = et - st
print('CPU Execution time:', res, 'seconds')

Loss after epoch 0: 121608.4765625
Loss after epoch 1: 120697.3828125
Loss after epoch 2: 115568.421875
Loss after epoch 3: 116181.125
Loss after epoch 4: 112804.21875
Loss after epoch 5: 107318.9375
Loss after epoch 6: 106973.375
Loss after epoch 7: 105189.8125
Loss after epoch 8: 103975.875
Loss after epoch 9: 97907.125
Loss after epoch 10: 93641.125
Loss after epoch 11: 93583.5
Loss after epoch 12: 90504.375
Loss after epoch 13: 90902.375
Loss after epoch 14: 90940.5
Loss after epoch 15: 88744.625
Loss after epoch 16: 87693.5
Loss after epoch 17: 85753.0
Loss after epoch 18: 85000.875
Loss after epoch 19: 85435.875
Loss after epoch 20: 83783.75
Loss after epoch 21: 80291.0
Loss after epoch 22: 78413.75
Loss after epoch 23: 78339.0
Loss after epoch 24: 78130.5
Loss after epoch 25: 78974.5
Loss after epoch 26: 77838.5
Loss after epoch 27: 77944.25
Loss after epoch 28: 77286.25
Loss after epoch 29: 77839.0
Loss after epoch 30: 76176.25
Loss after epoch 31: 75859.0
Loss after epoch 32: 

### 4 - Ensayar

In [17]:
def get_similar(word, n=10):
    print(f'Palabras relacionadas a {word}: ')
    for i in w2v_model.wv.most_similar(positive=[word], topn=n):
        print(i)

def get_disimilar(word, n=10):
    print(f'Palabras no relacionadas a {word}: ')
    for i in w2v_model.wv.most_similar(negative=[word], topn=n):
        print(i)

In [11]:
# Palabras que MÁS se relacionan con...:
get_similar('corrupción')

Palabras relacionadas a corrupción: 
('narcotráfico', 0.3547664284706116)
('traición', 0.34738820791244507)
('llamaron', 0.33839496970176697)
('lawfare', 0.2937644124031067)
('patagónica', 0.2871735692024231)
('gracia', 0.2871460020542145)
('petersen', 0.28226080536842346)
('exitoso', 0.2811082601547241)
('enriquecimiento', 0.27626824378967285)
('millonarios', 0.2748459279537201)


In [12]:
get_similar('inflación')

Palabras relacionadas a inflación: 
('déficit', 0.3705843985080719)
('conservadora', 0.30131256580352783)
('creían', 0.28993985056877136)
('vulnerables', 0.28245580196380615)
('propaganda', 0.281095951795578)
('sinceramente', 0.2784351408481598)
('estratégico', 0.2766098082065582)
('respaldo', 0.2730523347854614)
('creyeron', 0.2718909978866577)
('financiar', 0.2718221843242645)


In [13]:
get_similar('deuda')

Palabras relacionadas a deuda: 
('externa', 0.3655526340007782)
('soberana', 0.34475165605545044)
('93', 0.3107623755931854)
('deudas', 0.2966533899307251)
('soberanas', 0.2916410267353058)
('default', 0.28511807322502136)
('inversiones', 0.28235459327697754)
('extraordinaria', 0.2802611291408539)
('previos', 0.2686523199081421)
('constructoras', 0.2635423243045807)


In [14]:
get_similar('bancos')

Palabras relacionadas a bancos: 
('financieras', 0.39414650201797485)
('protección', 0.3189871609210968)
('militares', 0.3085186779499054)
('mejores', 0.30225899815559387)
('extranjeras', 0.29430150985717773)
('ganar', 0.2937650680541992)
('cadenas', 0.2872729003429413)
('1930', 0.2860015332698822)
('petersen', 0.2809734642505646)
('innovación', 0.28004899621009827)


In [20]:
get_similar('pobres')

Palabras relacionadas a pobres: 
('quiere', 0.32908549904823303)
('ricos', 0.27918875217437744)
('tus', 0.2776716649532318)
('necesitan', 0.2774547040462494)
('suficiente', 0.2757527232170105)
('pañuelos', 0.2750760018825531)
('invertido', 0.27338719367980957)
('convencer', 0.2723586857318878)
('hoteles', 0.27224239706993103)
('serlo', 0.27206623554229736)


In [15]:
# Palabras que MENOS se relacionan con...:
get_disimilar('juicio')

Palabras no relacionadas a juicio: 
('miseria', 0.11490596830844879)
('llevaba', 0.08340787142515182)
('infamia', 0.06398222595453262)
('entender', 0.06361836194992065)
('operar', 0.05474648252129555)
('votar', 0.05016825720667839)
('mayores', 0.04659157246351242)
('historias', 0.042836084961891174)
('obsesión', 0.042288489639759064)
('tituló', 0.03762070834636688)


In [ ]:
# Hacer regla de causa
# pobres es a ricos lo que ....

### 5 - Visualizar agrupación de vectores

In [16]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [19]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show() # esto para plotly en colab

/Users/marcosb/opt/miniconda3/envs/nlp_tensor/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/Users/marcosb/opt/miniconda3/envs/nlp_tensor/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

Algunas conclusiones del gráfico:
- Nisman y Fiscal, Mauricio y cambiemos y 2015, 